In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_parquet('/kaggle/input/original-data/original.parquet')

In [ ]:
data.head()

**Hover Duration Features Mean (session_id, event)** 

In [ ]:
# Hover duration by session id and event
d1 = data[['session_id','level_group','hover_duration','event_name']]

In [ ]:
d1 = d1[d1['hover_duration'].notnull()]

In [ ]:
hover_mean = d1.groupby(by=['session_id','level_group','event_name'])['hover_duration'].mean()

In [ ]:
map = hover_mean.xs('map_hover', level='event_name').reset_index()
object = hover_mean.xs('object_hover', level='event_name').reset_index()

In [ ]:
map.rename(columns={'hover_duration': 'map_hover_duration'}, inplace=True)
object.rename(columns={'hover_duration': 'object_hover_duration'}, inplace=True)

In [ ]:
d1 = pd.merge(map,object,on=['session_id','level_group'])

In [ ]:
d1.head()

**Bingo Features**

In [ ]:
d2 = data[['session_id','level_group','text_fqid','elapsed_time']]
d2 = d2[d2['text_fqid'].notnull()]

In [ ]:
d2 = d2[d2['text_fqid'].str.endswith('bingo')]

In [ ]:
first_bingo = d2.groupby(['session_id','level_group']).first().reset_index()

In [ ]:
first_bingo_elapsed_time = first_bingo[['session_id','level_group','elapsed_time']]

In [ ]:
first_bingo_elapsed_time.head()

In [ ]:
d2['time_diff']=d2['elapsed_time'].diff()

In [ ]:
d2 = d2[d2['time_diff'] >= 0]

In [ ]:
bingo_mean = d2.groupby(by=['session_id','level_group'])['time_diff'].mean().to_frame()

In [ ]:
bingo_mean.rename(columns={'time_diff': 'bingo_time_mean'}, inplace=True)

In [ ]:
d2 = pd.merge(bingo_mean,first_bingo_elapsed_time,on=['session_id','level_group'])

In [ ]:
d2.rename(columns={'elapsed_time': 'first_bingo_elapsed_time'}, inplace=True)

In [ ]:
d2.head()

**Room Change Feature: Ratio of room change per session id**

In [ ]:
d3 = data[['session_id','level_group','room_fqid']]

In [ ]:
d3['shift'] = d3['room_fqid'].shift()

In [ ]:
d3 = d3.groupby(['session_id','level_group']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

In [ ]:
d3 = d3[d3['shift'].notnull()]

In [ ]:
d3['room_change']= np.where(d3['room_fqid']==d3['shift'],0,1)

In [ ]:
d3_count = d3.groupby(by = ['session_id','level_group'])['room_change'].sum().to_frame()

In [ ]:
d3_count.head()

In [ ]:
d3_total = d3.groupby(by=['session_id','level_group'])['room_change'].count().to_frame()

In [ ]:
d3 = pd.merge(d3_count,d3_total,on=['session_id','level_group'])

In [ ]:
d3['room_change_ratio']=d3['room_change_x']/d3['room_change_y']

In [ ]:
d3.reset_index(inplace = True)

In [ ]:
d3.head()

In [ ]:
d3 = d3[['session_id','level_group','room_change_ratio']]

In [ ]:
d3.head()

**text_fqid change feature: Ratio of text_fqid change per session id**

In [ ]:
d4 = data[['session_id','level_group','text_fqid']]

In [ ]:
d4 = d4[d4['text_fqid'].notnull()]

In [ ]:
d4['shift'] = d4['text_fqid'].shift()

In [ ]:
d4 = d4.groupby(['session_id','level_group']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

In [ ]:
d4 = d4[d4['shift'].notnull()]

In [ ]:
d4['text_change']= np.where(d4['text_fqid']==d4['shift'],0,1)

In [ ]:
d4_count = d4.groupby(by = ['session_id','level_group'])['text_change'].sum().to_frame()
d4_total = d4.groupby(by= ['session_id','level_group'])['text_change'].count().to_frame()

In [ ]:
d4 = pd.merge(d4_count,d4_total,on=['session_id','level_group'])

In [ ]:
d4['text_change_ratio']=d4['text_change_x']/d4['text_change_y']

In [ ]:
d4=d4['text_change_ratio'].reset_index()

In [ ]:
d4.head()

Combine All Features

In [ ]:
d2.head()

In [ ]:
from functools import reduce
dfs = [d1, d3, d4]
# Merging all DataFrames
features = reduce(lambda left, right: pd.merge(left, right, on=['session_id','level_group'], how='inner'), dfs)

In [ ]:
features = pd.merge(features,d2,on=['session_id','level_group'],how='left')

In [ ]:
features.head(10)

In [ ]:
features.to_csv('features_keyue.csv', index=False)

In [ ]:
features.head()